### 0. Imports and requirements

In [ ]:
! pip install mambapy

In [1]:
! git clone https://github.com/SmirnovValeriy/dl-fintech-bki.git

Cloning into 'dl-fintech-bki'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 26 (delta 2), reused 4 (delta 0), pack-reused 16
Receiving objects: 100% (26/26), 16.23 MiB | 10.34 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [2]:
! wget https://storage.yandexcloud.net/ds-ods/files/materials/3c36dbad/train_target.csv
! wget https://storage.yandexcloud.net/ds-ods/files/materials/f020b5d6/train_data.zip
! wget https://storage.yandexcloud.net/ds-ods/files/materials/f2d2379a/test_data.zip

--2024-08-10 11:51:59--  https://storage.yandexcloud.net/ds-ods/files/materials/3c36dbad/train_target.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28888898 (28M) [text/csv]
Saving to: ‘train_target.csv’

train_target.csv    100%[===================>]  27.55M  36.3MB/s    in 0.8s    

2024-08-10 11:52:01 (36.3 MB/s) - ‘train_target.csv’ saved [28888898/28888898]

--2024-08-10 11:52:01--  https://storage.yandexcloud.net/ds-ods/files/materials/f020b5d6/train_data.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 319405083 (305M) [application/x-zip-compressed]
Saving to: ‘train_data.zip’


In [3]:
! unzip /content/train_data.zip
! unzip /content/test_data.zip

Archive:  /content/train_data.zip
  inflating: train_data/train_data_0.pq  
  inflating: train_data/train_data_1.pq  
  inflating: train_data/train_data_10.pq  
  inflating: train_data/train_data_11.pq  
  inflating: train_data/train_data_2.pq  
  inflating: train_data/train_data_3.pq  
  inflating: train_data/train_data_4.pq  
  inflating: train_data/train_data_5.pq  
  inflating: train_data/train_data_6.pq  
  inflating: train_data/train_data_7.pq  
  inflating: train_data/train_data_8.pq  
  inflating: train_data/train_data_9.pq  
Archive:  /content/test_data.zip
  inflating: test_data/test_data_0.pq  
  inflating: test_data/test_data_1.pq  


In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import sys
import pickle
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
import tqdm

import matplotlib.pyplot as plt

from dataclasses import dataclass

from mambapy.mamba import Mamba, MambaConfig


pd.set_option("display.max_columns", None)
plt.style.use("seaborn-pastel")

# Add the parent directory that will contain all necessary useful functions for data processing
sys.path.append("../")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

cuda


<ipython-input-1-70770cdcb8fd>:19: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-pastel")


### 1. Data Preprocessing

In [2]:
TRAIN_DATA_PATH = "/train_data/"
TEST_DATA_PATH = "/test_data/"

TRAIN_TARGET_PATH = "/train_target.csv"

In [3]:
train_target = pd.read_csv(TRAIN_TARGET_PATH)

In [4]:
%cd /dl-fintech-bki
from utils import read_parquet_dataset_from_local
from rnn_baseline.dataset_preprocessing_utils import features, transform_credits_to_sequences, create_padded_buckets

/content/dl-fintech-bki


In [ ]:
# Statistics on training and test sets
%%time
from collections import defaultdict

train_lens = []
test_lens = []
uniques = defaultdict(set)

for step in tqdm.notebook.tqdm(range(0, 12),
                     desc="Count statistics on train data"):
        credits_frame = read_parquet_dataset_from_local(TRAIN_DATA_PATH, step, 1, verbose=True)
        seq_lens = credits_frame.groupby("id").agg(seq_len=("rn", "max"))["seq_len"].values
        train_lens.extend(seq_lens)
        credits_frame.drop(columns=["id", "rn"], inplace=True)
        for feat in credits_frame.columns.values:
            uniques[feat] = uniques[feat].union(credits_frame[feat].unique())
train_lens = np.hstack(train_lens)

for step in tqdm.notebook.tqdm(range(0, 2, 2),
                     desc="Count statistics on test data"):
        credits_frame = read_parquet_dataset_from_local(TEST_DATA_PATH, step, 2, verbose=True)
        seq_lens = credits_frame.groupby("id").agg(seq_len=("rn", "max"))["seq_len"].values
        test_lens.extend(seq_lens)
        credits_frame.drop(columns=["id", "rn"], inplace=True)
        for feat in credits_frame.columns.values:
            uniques[feat] = uniques[feat].union(credits_frame[feat].unique())
test_lens = np.hstack(test_lens)
uniques = dict(uniques)

In [ ]:
keys_ = list(range(1, 59))
lens_ = list(range(1, 41)) + [45] * 5 + [50] * 5 + [58] * 8
bucket_info = dict(zip(keys_, lens_))

In [12]:
#@title Utility functions
def pad_sequence(array: np.ndarray, max_len: int) -> np.ndarray:
    """
    Принимает на вход массив массивов ``array`` и производит padding каждого вложенного массива до ``max_len``.

    Параметры:
    -----------
    array: numpy.ndarray
        Входной массив массивов.
    max_len: int
        Длина, до которой нужно сделать padding вложенных массивов.

    Возвращаемое значение:
    ----------------------
    output: numpy.ndarray
        Выходной массив.
    """
    if isinstance(max_len, float):
        print(max_len)
    output = np.zeros((len(features), max_len))
    output[:, :array.shape[1]] = array
    return output


def truncate(x, num_last_credits: int = 0):
    return pd.Series({"sequences": x.values.transpose()[:, -num_last_credits:]})

def create_padded_buckets(
        frame_of_sequences: pd.DataFrame,
        bucket_info: dict,
        save_to_file_path: str = None,
        has_target: bool = True
):
    """
    Реализует Sequence Bucketing технику для обучения рекуррентных нейронных сетей.
    Принимает на вход датафрейм ``frame_of_sequences`` с двумя столбцами: "id", "sequences"
    (результат работы функции transform_credits_to_sequences),
    словарь ``bucket_info``, где для последовательности каждой длины указано, до какой максимальной длины нужно делать
    padding, группирует кредиты по бакетам (на основе длины), производит padding нулями и сохраняет результат
    в pickle файл, если требуется.

    Параметры:
    -----------
    frame_of_sequences: pandas.DataFrame
        Входной датафрейм с двумя столбцами "id", "sequences" (результат работы функции transform_credits_to_sequences).
    bucket_info: Dict[int, int]
        Cловарь, где для последовательности каждой длины указано, до какой максимальной длины нужно делать padding.
    save_to_file_path: str, default=None
        Опциональный путь до файла, куда нужно сохранить результат. Если None, то сохранение не требуется.
    has_target: bool, deafult=True
        Флаг, есть ли в frame_of_sequences целевая переменная или нет. Если есть, то она также будет записана в выходной словарь.

    Возвращаемое значение:
    ----------------------
    dict_result: dict
        Выходной словарь со ключами:  "id", "padded_sequences", "target".
    """
    frame_of_sequences["sequence_length"] = frame_of_sequences["sequences"].apply(lambda x: len(x[1]))
    frame_of_sequences["bucket_idx"] = frame_of_sequences["sequence_length"].map(bucket_info)
    padded_seq = []
    targets = []
    ids = []

    for size, bucket in tqdm.tqdm(frame_of_sequences.groupby("bucket_idx"), desc="Extracting buckets"):
        padded_sequences = bucket["sequences"].apply(lambda x: pad_sequence(x, size)).values
        padded_seq.append(np.stack(padded_sequences, axis=0))

        if has_target:
            targets.append(bucket["flag"].values)

        ids.append(bucket["id"].values)

    frame_of_sequences.drop(columns=["bucket_idx"], inplace=True)

    dict_result = {
        "id": np.array(ids, dtype=object),
        "padded_sequences": np.array(padded_seq, dtype=object),
        "target": np.array(targets, dtype=object) if targets else []
    }

    if save_to_file_path:
        with open(save_to_file_path, "wb") as f:
            pickle.dump(dict_result, f)

def create_buckets_from_credits(path_to_dataset, bucket_info, save_to_path, frame_with_ids = None,
                                num_parts_to_preprocess_at_once: int = 1,
                                num_parts_total=50, has_target=False):
    block = 0
    for step in tqdm.notebook.tqdm(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                     desc="Preparing credit data"):
        credits_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once, verbose=True)
        credits_frame.loc[:, features] += 1
        seq = transform_credits_to_sequences(credits_frame)
        print("Transforming credits to sequences is done.")

        if frame_with_ids is not None:
            seq = seq.merge(frame_with_ids, on="id")

        block_as_str = str(block)
        if len(block_as_str) == 1:
            block_as_str = "00" + block_as_str
        else:
            block_as_str = "0" + block_as_str

        processed_fragment =  create_padded_buckets(seq, bucket_info=bucket_info, has_target=has_target,
                                                    save_to_file_path=os.path.join(save_to_path,
                                                                                   f"processed_chunk_{block_as_str}.pkl"))
        block += 1

In [14]:
# Split the data into training and validation particions. We will use 10% for validation
train, val = train_test_split(train_target, random_state=42, test_size=0.1)
print(f"Number of train samples: {train.shape[0]}, Number of validation samples: {val.shape[0]}")

((2700000, 2), (300000, 2))

In [15]:
TRAIN_BUCKETS_PATH = "/content/dl-fintech-bki/train_buckets_rnn"
VAL_BUCKETS_PATH = "/content/dl-fintech-bki/val_buckets_rnn"
TEST_BUCKETS_PATH = "/content/dl-fintech-bki/test_buckets_rnn"

In [ ]:
for buckets_path in [TRAIN_BUCKETS_PATH, VAL_BUCKETS_PATH, TEST_BUCKETS_PATH]:
    !rm -r $buckets_path
    !mkdir $buckets_path

Data preprocessing (takes around 30-40 minutes)

In [ ]:
%%time
create_buckets_from_credits(
    TRAIN_DATA_PATH,
    bucket_info=bucket_info,
    save_to_path=TRAIN_BUCKETS_PATH,
    frame_with_ids=train,
    num_parts_to_preprocess_at_once=1,
    num_parts_total=12, has_target=True
)

dataset_train = sorted([os.path.join(TRAIN_BUCKETS_PATH, x) for x in os.listdir(TRAIN_BUCKETS_PATH)])

In [ ]:
%%time
create_buckets_from_credits(TRAIN_DATA_PATH,
                            bucket_info=bucket_info,
                            save_to_path=VAL_BUCKETS_PATH,
                            frame_with_ids=val,
                            num_parts_to_preprocess_at_once=1,
                            num_parts_total=12, has_target=True)

dataset_val = sorted([os.path.join(VAL_BUCKETS_PATH, x) for x in os.listdir(VAL_BUCKETS_PATH)])

In [ ]:
%%time
create_buckets_from_credits(TEST_DATA_PATH,
                            bucket_info=bucket_info,
                            save_to_path=TEST_BUCKETS_PATH, num_parts_to_preprocess_at_once=1,
                            num_parts_total=2)

dataset_test = sorted([os.path.join(TEST_BUCKETS_PATH, x) for x in os.listdir(TEST_BUCKETS_PATH)])

### 2. Modeling

In [19]:
from rnn_baseline.data_generators import batches_generator
from rnn_baseline.pytorch_training import train_epoch, eval_model, inference
from rnn_baseline.training_aux import EarlyStopping

In [20]:
def compute_embed_dim(n_cat: int) -> int:
    return min(600, round(1.6 * n_cat**0.56))

embedding_projections = {feat: (max(uniq)+1, compute_embed_dim(max(uniq)+1)) for feat, uniq in uniques.items()}

In [ ]:
class CreditsRNN(nn.Module):
    def __init__(
        self,
        features,
        embedding_projections,
        rnn_units=128,
        top_classifier_units=256,
        dropout_p=0.05,
    ):
        super(CreditsRNN, self).__init__()
        self.credits_cat_embeddings = nn.ModuleList([self.create_embedding_projection(*embedding_projections[feature])
                                                      for feature in features])

        self.dropout2d = nn.Dropout2d(dropout_p)

        config = MambaConfig(
            d_model=sum([embedding_projections[x][1] for x in features]),
            n_layers=1,
        )
        self.mamba = Mamba(config)

        self.hidden_size = rnn_units * 4

        self.dropout1d = nn.Dropout(dropout_p)

        self.head = nn.Sequential(
            nn.Linear(in_features=self.hidden_size, out_features=top_classifier_units),
            nn.SiLU(),
            nn.Linear(in_features=top_classifier_units, out_features=1)
        )

    def forward(self, features):
        embeddings = [embedding(features[i]) for i, embedding in enumerate(self.credits_cat_embeddings)]
        concated_embeddings = self.dropout2d(torch.cat(embeddings, dim=-1))

        output = self.mamba(concated_embeddings)

        output_max_pool = output.max(dim=1)[0]
        output_avg_pool = output.sum(dim=1) / output.shape[1]

        combined_input = torch.cat([output_max_pool, output_avg_pool], dim=-1)
        combined_input = self.dropout1d(combined_input)

        raw_output = self.head(combined_input)

        return raw_output

    @classmethod
    def create_embedding_projection(cls, cardinality, embed_size, add_missing=True, padding_idx=0):
        add_missing = 1 if add_missing else 0
        return nn.Embedding(num_embeddings=cardinality + add_missing, embedding_dim=embed_size, padding_idx=padding_idx)

### 3. Training

In [33]:
!rm -rf ./checkpoints/
!mkdir ./checkpoints/

In [ ]:
!rm -r ./checkpoints/pytorch_baseline
!mkdir ./checkpoints/pytorch_baseline

In [35]:
path_to_checkpoints = "./checkpoints/pytorch_baseline/"
es = EarlyStopping(patience=3, mode="max", verbose=True, save_path=os.path.join(path_to_checkpoints, "best_checkpoint.pt"),
                   metric_name="ROC-AUC", save_format="torch")

In [36]:
num_epochs = 10
train_batch_size = 128
val_batch_size = 128

In [ ]:
model = CreditsRNN(
    features,
    embedding_projections,
).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"Sum of model parameters: {total_params}")

In [38]:
model

CreditsRNN(
  (credits_cat_embeddings): ModuleList(
    (0): Embedding(21, 8, padding_idx=0)
    (1): Embedding(19, 7, padding_idx=0)
    (2): Embedding(19, 8, padding_idx=0)
    (3-4): 2 x Embedding(18, 8, padding_idx=0)
    (5): Embedding(17, 8, padding_idx=0)
    (6): Embedding(21, 9, padding_idx=0)
    (7): Embedding(9, 5, padding_idx=0)
    (8): Embedding(7, 4, padding_idx=0)
    (9): Embedding(3, 2, padding_idx=0)
    (10): Embedding(5, 3, padding_idx=0)
    (11): Embedding(15, 7, padding_idx=0)
    (12): Embedding(19, 8, padding_idx=0)
    (13): Embedding(21, 9, padding_idx=0)
    (14): Embedding(11, 6, padding_idx=0)
    (15): Embedding(7, 4, padding_idx=0)
    (16): Embedding(21, 9, padding_idx=0)
    (17-21): 5 x Embedding(3, 2, padding_idx=0)
    (22-24): 3 x Embedding(21, 9, padding_idx=0)
    (25-27): 3 x Embedding(3, 2, padding_idx=0)
    (28-38): 11 x Embedding(5, 3, padding_idx=0)
    (39): Embedding(6, 4, padding_idx=0)
    (40-47): 8 x Embedding(5, 3, padding_idx=0)
 

In [39]:
optimizer = torch.optim.AdamW(lr=1e-3, params=model.parameters())

In [ ]:
%%time
for epoch in range(num_epochs):
    print(f"Starting epoch {epoch+1}")
    train_epoch(model, optimizer, dataset_train, batch_size=train_batch_size,
                shuffle=True, print_loss_every_n_batches=500, device=device)

    val_roc_auc = eval_model(model, dataset_val, batch_size=val_batch_size, device=device)
    es(val_roc_auc, model)

    if es.early_stop:
        print("Early stopping reached. Stop training...")
        break
    torch.save(model.state_dict(), os.path.join(path_to_checkpoints, f"epoch_{epoch+1}_val_{val_roc_auc:.3f}.pt"))

    print(f"Epoch {epoch+1} completed. Val ROC AUC: {val_roc_auc}")